In [1]:
import os
import sys
import re

SPARK_HOME = os.environ['SPARK_HOME']
sys.path.insert(0, os.path.join(SPARK_HOME, "python", "build"))
sys.path.insert(0, os.path.join(SPARK_HOME, "python"))

In [2]:
from pyspark import  SparkContext

sc = SparkContext( 'local', 'pyspark')
textFile = sc.textFile("aaa") #测试文本
textFile.count()

116710

# 用map和reduce实现collect

In [3]:
linesWithResearch = textFile.filter(lambda line: "Marriage" in line)
linesWithResearch.collect()

['Social Norms About Desires for Marriage and Children and Anticipated Chore Participation,”',
 '2. Scott S. Hall and Shelley M. MacDermid, “A Typology of Dual Earner Marriages Based on',
 'That Inhibit Greater Father Involvement in Family Work,” Journal of Marriage and Family 61, no. 1',
 'Social Embeddedness of Routine Family Work,” Journal of Marriage and Family 62, no. 4 (2000):',
 '“Control Over Money in Marriage,” in Marriage and the Economy: Theory and Evidence from',
 'Neil Chethik, Voice Male: What Husbands Really Think About Their Marriages, Their Wives, Sex,',
 '10. Sreedhari D. Desai, Dolly Chugh, and Arthur Brief, “Marriage Structure and Resistance to the',
 'Social Norms About Desires for Marriage and Children and Anticipated Chore Participation,”',
 '2. Scott S. Hall and Shelley M. MacDermid, “A Typology of Dual Earner Marriages Based on',
 'That Inhibit Greater Father Involvement in Family Work,” Journal of Marriage and Family 61, no. 1',
 'Social Embeddedness of Routin

In [4]:
linesWithResearch.map(lambda x: [x]).reduce(lambda a, b: a + b)

['Social Norms About Desires for Marriage and Children and Anticipated Chore Participation,”',
 '2. Scott S. Hall and Shelley M. MacDermid, “A Typology of Dual Earner Marriages Based on',
 'That Inhibit Greater Father Involvement in Family Work,” Journal of Marriage and Family 61, no. 1',
 'Social Embeddedness of Routine Family Work,” Journal of Marriage and Family 62, no. 4 (2000):',
 '“Control Over Money in Marriage,” in Marriage and the Economy: Theory and Evidence from',
 'Neil Chethik, Voice Male: What Husbands Really Think About Their Marriages, Their Wives, Sex,',
 '10. Sreedhari D. Desai, Dolly Chugh, and Arthur Brief, “Marriage Structure and Resistance to the',
 'Social Norms About Desires for Marriage and Children and Anticipated Chore Participation,”',
 '2. Scott S. Hall and Shelley M. MacDermid, “A Typology of Dual Earner Marriages Based on',
 'That Inhibit Greater Father Involvement in Family Work,” Journal of Marriage and Family 61, no. 1',
 'Social Embeddedness of Routin

# 用reduce实现max

In [5]:
linesWithResearch.max()

'“Control Over Money in Marriage,” in Marriage and the Economy: Theory and Evidence from'

In [6]:
linesWithResearch.reduce(lambda a, b: a if a > b else b)

'“Control Over Money in Marriage,” in Marriage and the Economy: Theory and Evidence from'

# 统计单词个数，从大到小排序，并过滤掉长度为1的单词，大小写不敏感

In [7]:
%timeit textFile.flatMap(lambda line: re.split(' |\n|\[|\]|\(|\)|/|#|\.|\+|\-', str.lower(line)))\
                .filter(lambda word: len(word) > 1)\
                .map(lambda word: (word, 1))\
                .reduceByKey(lambda a, b: a + b)\
                .sortBy(lambda x: x[1], ascending=False)\
                .collect()

1 loop, best of 3: 2.09 s per loop


# python版统计单词个数

In [8]:
textContent = textFile.collect()

In [9]:
def countWord(textContent):
    rst = {}
    for line in textContent:
        words = re.split(' |\n|\[|\]|\(|\)|/|#|\.|\+|\-', str.lower(line))
        for word in words:
            if len(word) <= 1:
                continue
            if word in rst:
                rst[word] = rst[word] + 1
            else:
                rst[word] = 1
    
    return sorted(rst.items(), key = lambda item: item[1], reverse=True)

In [10]:
%timeit countWord(textContent)

1 loop, best of 3: 947 ms per loop
